# Dask

Dask erfüllt zwei verschiedene Aufgaben:

1. die dynamische Aufgabenplanung wird optimiert, ähnlich wie bei [Airflow](https://airflow.apache.org/), [Luigi](https://github.com/spotify/luigi) oder [Celery](https://docs.celeryproject.org/)
2. Arrays, Dataframes und Lists werden parallel mit dynamischem Task Scheduling ausgeführt.

## Skalierung von Laptops bis hin zu Clustern


Dask kann mit pipenv einfach auf einem Laptop installiert werden und erweitert die Größe der Datensätze von *passt in den Arbeitsspeicher* zu *passt auf die Festplatte*. Dask kann jedoch auch auf einen Cluster mit Hunderten von Rechnern skaliert werden. Dask ist robust, flexibel, Data Local und hat eine geringe Latenzzeit. Weitere Informationen findet ihr in der Dokumentation zum [Distributed Scheduler](https://distributed.dask.org/en/latest/). Dieser einfache Übergang zwischen einer einzelnen Maschine und einem Cluster ermöglicht einen einfachen Start und ein Wachstum nach Bedarf.

## Dask installieren


Ihr könnt alles installieren, was für die meisten gängigen Anwendungen von Dask erforderlich ist (Arrays, Dataframes, …). Dies installiert sowohl Dask als auch Abhängigkeiten wie NumPy, Pandas, usw., die für verschiedene Arbeiten benötigt werden:

``` bash
$ pipenv install "dask[complete]"
```

Es können aber auch nur einzelne Subsets installiert werden:

``` bash
$ pipenv install "dask[array]"
$ pipenv install "dask[dataframe]"
$ pipenv install "dask[diagnostics]"
$ pipenv install "dask[distributed]"
```

### Testen der Installation

In [1]:
!pytest   ../../../spackenvs/python-374/.spack-env/view/lib/python3.7/site-packages/dask/tests  ../../../spackenvs/python-374/.spack-env/view/lib/python3.7/site-packages/dask/array/tests

============================= test session starts ==============================
platform darwin -- Python 3.7.4, pytest-6.2.5, py-1.10.0, pluggy-1.0.0
Users/veit
plugins: anyio-3.3.1
collected 4218 items / 16 skipped / 4202 selected                              
…
=============================== warnings summary ===============================
…
-- Docs: https://docs.pytest.org/en/stable/warnings.html
==== 4013 passed, 207 skipped, 14 xfailed, 22 warnings in 357.82s (0:05:57) ====


## Vertraute Bedienung

### Dask DataFrame

… imitiert Pandas

In [2]:
import pandas as pd
df = pd.read_csv('2021-09-01.csv')
df.groupby(df.user_id).value.mean()

In [3]:
import dask.dataframe as dd
dd = pd.read_csv('2021-09-01.csv')
dd.groupby(dd.user_id).value.mean()

> **Siehe auch:**
> 
> * [Dask DataFrame Docs](https://docs.dask.org/en/latest/dataframe.html)
> * [Dask DataFrame Best Practices](https://docs.dask.org/en/latest/dataframe-best-practices.html)

### Dask Array

… imitiert NumPy

In [4]:
import numpy as np
f = h5py.File('mydata.h5')
x = np.array(f['.'])

In [5]:
import dask.array as da
f = h5py.File('mydata.h5')
x = da.array(f['.'])

> **Siehe auch:**
> 
> * [Dask Array Docs](https://docs.dask.org/en/latest/array.html)
> * [Dask Array Best Practices](https://docs.dask.org/en/latest/array-best-practices.html)

### Dask Bag

… imitiert [iterators](https://docs.python.org/3/library/itertools.html), [Toolz](https://toolz.readthedocs.io/en/latest/index.html) und [PySpark](http://spark.apache.org/docs/latest/api/python/).

In [6]:
import dask.bag as db
import json
b = db.read_text('2021-09-01.csv').map(json.loads)
b.pluck('user_id').frequencies().topk(10, lambda pair: pair[1]).compute()

> **Siehe auch:**
> 
> * [Dask Bag Docs](https://docs.dask.org/en/latest/bag.html)

### Dask Delayed

… imitiert loops und umschließt benutzerdefinierten Code.

In [7]:
from dask import delayed
L = []
for fn in '2021-*-*.csv':             # Schleife von Berechnungen
    data = delayed(load)(fn)          # Verzögern der auszuführenden Funktion
    L.append(delayed(process)(data))  # Verbindungen zwischen Variablen herstellen

result = delayed(summarize)(L)
result.compute()

> **Siehe auch:**
> 
> * [Dask Delayed Docs](https://docs.dask.org/en/latest/delayed.html)
> * [Dask Delayed Best Practices](https://docs.dask.org/en/latest/delayed-best-practices.html)
> * [Dask-Pipeline-Beispiel: Tracking der Internationalen Raumstation mit Dask](../../clean-prep/dask-pipeline.html)

## Das ``concurrent.futures``-Interface ermöglicht die Übermittlung von selbstdefinierten Aufgaben.

> **Anmerkung:**
> 
> Für das folgende Beispiel muss Dask mit der `distributed`-Option installiert werden, z.B.
> 
> ``` bash
> $ pipenv install "dask[distributed]"
> ```

In [8]:
from dask.distributed import Client
client = Client('scheduler:port')

futures = []
for fn in filenames:
    future = client.submit(load, fn)
    futures.append(future)

summary = client.submit(summarize, futures)
summary.result()

> **Siehe auch:**
> 
> * [Dask Futures Docs](https://docs.dask.org/en/latest/futures.html)
> * [Dask Futures Quickstart](https://distributed.dask.org/en/latest/quickstart.html)
> * [Dask Futures Examples](https://examples.dask.org/futures.html)